In [1]:
import re
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
def get_page_meta(url):
    if not "公告" in title and not "版規" in title: # 先做最基礎的判斷, 非公告和版規我回傳答案
        response = requests.get(article_url).text
        html = BeautifulSoup(response)
        content = html.find("div", id="main-content")
        result = {} # 準備我們要回傳的字典
        values = content.find_all("span", class_="article-meta-value")
        # 先把文章資訊記錄在字典裡
        result["author"] = values[0].text
        result["board"] = values[1].text
        result["title"] = values[2].text
        result["time"] = values[3].text
        
        meta = content.find_all("div", class_="article-metaline")
        for m in meta:
            m.extract()
        right_meta = content.find_all("div", class_="article-metaline-right")
        for single_meta in right_meta:
            single_meta.extract()

        pushes = content.find_all("div", class_="push")
        score = 0
        for single_push in pushes:
            pushtag = single_push.find("span", class_="push-tag").text
            if "推" in pushtag:
                score = score + 1
            elif "噓" in pushtag:
                score = score - 1
            single_push.extract()
        # 分數和內容
        result["score"] = score
        result["content"] = content.text
        return result
    else:# 公告和版規我就直接回傳 None
        return None

In [3]:
# 從 movie 版首頁開始
url = "https://www.ptt.cc/bbs/C_Chat/index.html"

In [4]:
# 準備要記錄的表格
df = pd.DataFrame(columns=["作者", "看板", "標題", "時間", "分數", "內容"])

In [5]:
# 走過五頁, range(5) 會幫你產生一個類似 [0, 1, 2, 3, 4] 的 list
for times in range(5):
    response = requests.get(url).text
    html = BeautifulSoup(response)
    articles = html.find_all("div", class_="r-ent")    # 得到每一篇文章的區域
    for single_article in articles:    # 走過每一篇文章
        title_area = single_article.find("div", class_="title").find("a")    # 得到 title 的超連結元素 <a>
        try:
            # 如果有 title 才繼續 (被刪除的文章會沒有 title)
            title = title_area.contents[0]    # 得到 title 的文字
            article_url = "https://www.ptt.cc" + title_area["href"]    # 得到 title 的超連結屬性 href
            
            result = get_page_meta(article_url)    # 使用我們剛剛定義的函式
            data = [result["author"], result["board"], result["title"], result["time"], result["score"], result["content"]]
            s = pd.Series(data, index=["作者", "看板", "標題", "時間", "分數", "內容"])
            df = df.append(s, ignore_index=True)
        except:
            continue
                
    # 往下一頁前進, string 參數可以找裡面文字符合我們帶入字串的元素
    url = "https://www.ptt.cc" + html.find("a", text=re.compile(r"上頁"))["href"]

In [6]:
df

,作者,看板,標題,時間,分數,內容
0,smith2012 (smith2012),C_Chat,[21春] vivy 01~05 觀後推薦,Sun Apr 25 23:26:35 2021,6,\n\n內文無雷，推文不保證\n\n真的是一部很有潛力的作品\n\n裡頭有時空跳躍和一些解謎...
1,AlisaRein (AlisaR),C_Chat,Re: [死神] 日番谷冬獅郎走錯了哪一步？被嘲諷到現在,Sun Apr 25 23:26:52 2021,1,\n\n就需要有一個角色來襯托敵人的強大\n\n例如說聖鬥士找金牛座當作每篇敵人開場必定去找...
2,a125g (醬油膏),C_Chat,[閒聊] 大空486穿這樣不會痛嗎?,Sun Apr 25 23:27:14 2021,6,\nHOLO的大空486\n\n假小子\n\n她有一套服裝是這樣\nhttps://i.im...
3,mizuarashi (米茲阿拉西),C_Chat,[閒聊] 雙生遊戲 第一話,Sun Apr 25 23:30:50 2021,7,\nyoung magazine新連載\n\nhttps://i.imgur.com/Xwg...
4,killeryuan (龍鳥),C_Chat,Re: [退隊] 遭到勇者驅逐的白魔導師，被S級冒險者撿走,Sun Apr 25 23:32:13 2021,2,\n我記得有另外一部很像的\n\n只是不是白魔 是藥師 做的藥又便宜又好用\n\n被勇者小隊...
...,...,...,...,...,...,...
91,a86425380 (瑞斯叔叔),C_Chat,[討論] 遊戲王zexal值得看嗎？,Sun Apr 25 21:51:40 2021,39,\n\n如題\n回鍋遊戲王DL\n\n最近看到DL居然已經出到超量了\n\n我的記憶還停在5...
92,slashmoon (網球丸子),C_Chat,[問題] 求推薦,Sun Apr 25 21:53:03 2021,13,\n想找戰鬥畫面不錯 劇情不要過度無腦的戰鬥番\n\n近期咒術很愛 十年內應該都可以\n\n...
93,Antihuman,C_Chat,[閒聊] 遊戲裡硬要加入其他小遊戲幹嘛,Sun Apr 25 21:53:55 2021,5,\n每個人喜歡的遊戲類型不同，在購買前就會考慮了\n\n然而很多遊戲除了核心內容之外，還要硬...
94,whe84311 (Rainsa),C_Chat,[閒聊] 有人拿刀闖進米哈游總部要砍老闆脖子,Sun Apr 25 21:54:49 2021,9,\n\n4月24日晚，網傳米哈游公司園區現場逮捕了一名欲持刀刺殺公司兩位創始人的歹徒。\n\...


In [7]:
df.to_csv("ptt_C_chat.csv", index=False, encoding="utf_8_sig")